In [ ]:
#In order to read the teacher observation note more effeciently. Practice a way to extracting, grouping the text data into information
#Referencing Source A: https://www.kaggle.com/code/tejashpopate/text-mining-in-python. Referencing Source B:https://towardsai.net/p/data-mining/text-mining-in-python-steps-and-examples-78b3f8fd913b
#Further mining the topic of the text: https://monkeylearn.com/topic-analysis/. Text Classification:https://realpython.com/python-keras-text-classification/
#Training Dataset: SchoolMintGrow QuickFeedback observation
#Training Maetrial : https://portal.smarterbalanced.org/library/en/ela-construct-relevant-vocabulary.pdf
#Testing Dataset: SchoolMintGrow Teacher Observation
#Project Roadmap:Stage 1, successfully tested on Glow - Glow with Word Cloud and LDA Topic Extracting on sampled data set
                #Stage 2, Test on Grow - Grow.
                #Stage 3, Test on full data set with Glow -Glow and Grow - Grow.
                #Stage 4, Test on survey response to replicate, optimize, clean the code book.
                #Stage 5, Continue build up the model with clustering model, the data set needs to be trained(Categorized).
                #Stage 6, Apply the trained clustered model to new testing data source to automatically assign the category by the machine. Quality check manually.
                

In [ ]:
!pip install textblob
!pip install vaderSentiment
!pip install textmining3
!pip3 install wordcloud
#import gensim
import pandas as pd
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
Analyzer = SentimentIntensityAnalyzer()
from nltk.corpus import stopwords
import string
import nltk
import textmining
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
from gensim.utils import simple_preprocess
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
# set the working dierectory

df = pd.read_excel('/Users/lipingsun/Desktop/wcpa/Teacher Evaluation/Feedback Activity.xlsx')
df['Glow - Glow']=df['Glow - Glow'].astype('str')

output='/Users/lipingsun/Desktop/wcpa/Teacher Evaluation/Output_SMG_tableau.xlsx'

In [ ]:
df[:5]

In [ ]:
import nltk
>>> nltk.download()
# define the stopwords and also import predefined ones
stop=set(stopwords.words("english"))
# Extract punctuation marks
punct_exclude=set(string.punctuation)

In [ ]:
from nltk.corpus import brown
>>> brown.words()

In [ ]:
# perform preoprocessing using function below
def clean(doc):
    stop_free=" ".join([i for i in doc.lower().split() if i not in stop])
    punct_free=''.join(ch for ch in stop_free if ch not in punct_exclude)
    num_free=''.join(i for i in punct_free if not i.isdigit())
    return num_free
####Column index 4(Glow to Glow). There is a problem of using the index 5(Grow to Grow) or 6(Action Steps) because there are some blank values.
####Need to address that either through the raw data file or through the data selection by eleminating the Nulls:dropping rows where 'Review Text' is null
#df.dropna(subset=['Grow - Grow'], inplace=True)
post_corpus=[clean(df.iloc[i,6]) for i in range(0,df.shape[0])] ###1/4Change the iloc to appropriate index number to switch


In [ ]:
print(type(post_corpus))
print(post_corpus[5])

In [ ]:
# create term document matrix
tdm=textmining.TermDocumentMatrix() # use a function from textmining library
for i in post_corpus:
    #print(i)
    tdm.add_doc(i)# update the matrix with each variable conversion

In [ ]:
type(tdm)

In [ ]:
output='/Users/lipingsun/Desktop/wcpa/Teacher Evaluation/Output_smg_tableau.xlsx'

In [ ]:
#write tdm into dataframe
tdm.write_csv("TDM_DataFrame.csv",cutoff= 1) #cutoff won't consider 1st line

In [ ]:
def buildMatrix(self,document_list):
        print("building matrix...")
        tdm = textmining.TermDocumentMatrix()
        for doc in document_list:
             tdm.add_doc(doc)
        #write tdm into dataframe
        tdm.write_csv(r'path\matrix.csv', cutoff=1)

In [ ]:
Result=pd.read_csv("TDM_DataFrame.csv")

In [ ]:
Result.head(20)

In [ ]:
df.shape

In [ ]:
Result.shape

In [ ]:
#plot wordcloud
wordcloud= WordCloud(width=1000,height=500, stopwords=STOPWORDS, background_color='white').generate(''.join(df['Grow - Grow'])) ###2/4Change the column name to appropriate one to switch
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
#Sentimental Analysis
FinalResult=pd.DataFrame()
for i in range(0,df.shape[0]):
    #print(i)
    blob=TextBlob(df.iloc[i,6]) ###3/4Change the iloc to appropriate index number to switch
    temp=pd.DataFrame({"Comments for Glow":df.iloc[i,6],"compound":blob.sentiment.polarity},index=[0])
    FinalResult=FinalResult.append(temp)

In [ ]:
print(FinalResult.shape)
FinalResult[:10]
vs= Analyzer.polarity_scores("Vader sentiment looks interesting, I have high hopes!")
print(vs)

In [ ]:

FinalResult.head()
FinalResult[:30]
FinalResult.describe()


In [ ]:
#Option 1: Follow the steps here to LDA Topic Modeling.
#https://www.analyticsvidhya.com/blog/2021/07/topic-modelling-with-lda-a-hands-on-introduction/?#h2_3
from nltk.corpus import stopwords  #stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer  
from sklearn.feature_extraction.text import TfidfVectorizer
stop_words=set(nltk.corpus.stopwords.words('english'))


def clean_text(headline):
      le=WordNetLemmatizer()
      word_tokens=word_tokenize(headline)
      tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
      cleaned_text=" ".join(tokens)
      return cleaned_text
df['cleaned_text']=df['Grow - Grow'].apply(clean_text)###4/4Change the column name to appropriate one to switch
df[:10]

vect =TfidfVectorizer(stop_words=stop_words,max_features=1000)
vect_text=vect.fit_transform(df['cleaned_text'])

from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=5,
learning_method='online',random_state=42,max_iter=1) #n_components = topic #
lda_top=lda_model.fit_transform(vect_text)
print("Document 18: ")
for i,topic in enumerate(lda_top[18]): 
  print("Topic ",i,": ",topic*100,"%")

vocab = vect.get_feature_names()
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10] #lda_top = # of words to describe the topics
     print("Topic "+str(i)+": ")
     for t in sorted_words:
            print(t[0],end=" ")
            print("n")
lda_top[:20]
#Glow
#Topic 0: Ways of suppurt to help students learn during the class(scratch pad, small group discussion). 
#Topic 1: Ways of instruction strategy to facilitate the engagement of students to solving the problem(EX. board/ask question).
#Topic 2: The learning environment and culture as well as classroom expectations improve the relationship.
#Topic 3: Small group, pause to check understanding, clear expectations are effective.
#Topic 4: Learning material and supplements setting are important to facilitate at grade-level learning.

#Grow(For empty row. I inserted "EMPTY" as a placeholder)
#Topic 0: Ways of suppurt to help students learn during the class(scratch pad, small group discussion). 
#Topic 1: Ways of instruction strategy to facilitate the engagement of students to solving the problem(EX. board/ask question).
#Topic 2: The learning environment and culture as well as classroom expectations improve the relationship.
#Topic 3: Small group, pause to check understanding, clear expectations are effective.
#Topic 4: Learning material and supplements setting are important to facilitate at grade-level learning.


In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=df, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('nCoherence Score: ', coherence_lda)

In [ ]:
#Evaluate the Topics and relate back to the texts
#https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
#https://storage.googleapis.com/destinationinsight.appspot.com/lda.html#topic=2&lambda=0&term=
#USE THIS ONE TO FURTHER INVESTIGATE THE CASE: https://www.kaggle.com/code/panks03/clustering-with-topic-modeling-using-lda

In [ ]:
for index, score in sorted(lda_top[df.['Glow - Glow'][2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_top.print_topic(index, 10)))

In [ ]:
# Write TOPICS Array data to excel file
writer = pd.ExcelWriter(output)

pd.DataFrame(lda_top).to_excel(writer, sheet_name='TopicScore',index=False)

writer.save()

In [ ]:
#Option 2: Follow the steps here to clustering the comments.
#https://www.kaggle.com/code/panks03/clustering-with-topic-modeling-using-lda


In [ ]:
#Option 3: Follow the steps here to LDA Topic Modeling.
#https://neptune.ai/blog/pyldavis-topic-modelling-exploration-tool-that-every-nlp-data-scientist-should-know
!pip3 install pyldavis
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import gensim.corpora as corpora
from gensim.corpora import distionary
from gensim.models.cohenrencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from pprint import pprint
import spacy
import pickle
import re
import matplotlib.pyplot as plt

# number of topics
num_topics = 10

# Visualize the topics
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

